In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from pandas import DataFrame
import uuid
import datetime
from skimage.transform import resize

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from registration_utilities import *
from annotation_utilities import interpolate_contours_to_volume

Setting environment for AWS compute node


/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No vtk


In [2]:
# def generate_annotation_file_from_aligned_atlas(stack_m, stack_f, classifier_setting_m, classifier_setting_f,
#                                                warp_setting, level=.5, sample_every=1):
#     """
#     Compute the structure contours on a fixed stack according to the atlas aligned to it.
#     The structure contours are stored as a pandas dataframe. 
#     Rows are section indices and columns are structure names.
#     Structure names are unsided only.
#     Also save this dataframe as a hdf file. 
#     The file location can be retrieved by DataManager.get_annotation_filepath().
    
#     Returns:
#         dataframe: the structure contours
#     """
        
# #     warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_m, stack_f=stack_f,
# #                                     classifier_setting_m=classifier_setting_m,
# #                                     classifier_setting_f=classifier_setting_f,
# #                                     warp_setting=warp_setting, sided=True)
# #     downsample_factor = 32
    
# #     xmin_vol_f, _, ymin_vol_f, _, zmin_vol_f, _ = \
# #     DataManager.load_original_volume_bbox(stack=stack_f, vol_type='score', structure='7N', downscale=downsample_factor, classifier_setting=classifier_setting_f)
    
# #     structure_contours = get_structure_contours_from_aligned_atlas(warped_volumes, downsample_factor=downsample_factor,
# #                                                          volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
# #                                                         sections=metadata_cache['valid_sections'][stack_f], 
# #                                                          level=level, sample_every=sample_every)

# #     del warped_volumes
    
# #     contours_entries = {}
# #     for sec, row in structure_contours.iteritems():
# #         for name_s, vertices in row.iteritems():
# #             id_ = str(uuid.uuid4().fields[-1])
# #             side = get_side_from_label(name_s)
# #             if side is None:
# #                 side = 'S'
# #             contour_entry = {
# #                 'name': convert_to_original_name(name_s),
# #                 'label_position': vertices.mean(axis=0),
# #                 'side': side,
# #                 'creator': 'auto',
# #                 'time_created': datetime.datetime.now().strftime("%m%d%Y%H%M%S"),
# #                 'edits': [],
# #                 'vertices': vertices,
# #                 'downsample': 1,
# #                 'flags': 1, #1 means contour is unconfirmed.
# #                 'section': sec,
# #                 'filename': metadata_cache['sections_to_filenames'][stack_f][sec],
# #                 'orientation': 'sagittal',
# #                 'parent_structure': [],
# #                 'side_manually_assigned': False,
# #                 'id': id_
# #             }
# #             contours_entries[id_] = contour_entry           
        
# #     contours_df = DataFrame(contours_entries).T
    
# #     contours_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
# #                                                            classifier_setting_m=classifier_setting_m,
# #                                                           classifier_setting_f=classifier_setting_f,
# #                                                           warp_setting=warp_setting, suffix='contours')
# #     save_hdf_v2(contours_df, contours_df_fp)
# #     upload_to_s3(contours_df_fp)
    
#     ####################################
#     ## Reconstruct structure volumes  ##
#     ####################################
    
#     contours_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
#                                                        classifier_setting_m=classifier_setting_m,
#                                                       classifier_setting_f=classifier_setting_f,
#                                                       warp_setting=warp_setting, suffix='contours')
#     download_from_s3(contours_df_fp)
#     contours_df = load_hdf_v2(contours_df_fp)
    
#     volume_downsample = 8
    
#     structure_entries = {}
#     for s in all_known_structures_sided:
#         name_u, side, _, _ = parse_label(s)
#         if side is None:
#             side = 'S'
        
#         contour_points_grouped_by_pos = {np.mean(DataManager.convert_section_to_z(stack='MD635', sec=p['section'], downsample=1)) / volume_downsample: 
#                                          p['vertices'] / volume_downsample
#                                          for _, p in contours_df[(contours_df['name'] == name_u) & (contours_df['side'] == side)].iterrows()}

#         if len(contour_points_grouped_by_pos) >= 2:
#             vol, bbox = interpolate_contours_to_volume(contour_points_grouped_by_pos, 'z')
#             id_ = str(uuid.uuid4().fields[-1])
#             structure_entry = {
#                 'name': name_u,
#                 'side': side,
#                 'volume_in_bbox': vol,
#                 'bbox': bbox,
#                 'downsample': volume_downsample,
#                 'creator': 'auto',
#                 'time_created': datetime.datetime.now().strftime("%m%d%Y%H%M%S"),
#                 'edits': []
#             }
#             structure_entries[id_] = structure_entry
#         else:
#             sys.stderr.write('Cannot interpolate because there are fewer than two contours for structure (%s, %s).\n' % (name_u, side))

#     structures_df = DataFrame(structure_entries).T
    
#     structures_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
#                                                            classifier_setting_m=classifier_setting_m,
#                                                           classifier_setting_f=classifier_setting_f,
#                                                           warp_setting=warp_setting, suffix='structures')
#     save_hdf_v2(structure_entries, structures_df_fp)
#     upload_to_s3(structures_df_fp)   
    

In [5]:
# generate_annotation_file_from_aligned_atlas(stack_m='atlasV3', stack_f='MD635', 
#                                             classifier_setting_m=classifier_id, 
#                                             classifier_setting_f=classifier_id, 
#                                             warp_setting=warp_setting,
#                                            level=.5, sample_every=1)

In [2]:
def rescale_binary_volume(vol, rescale_factor):
    """
    Rescale a binary volume.
    
    Args:
        vol (3D array of bool): binary volume.
        rescale_factor (int): 
        
    Returns:
        scaled_vol (3D array of bool): scaled volume in bounding box
        scaled_bbox (6-tuple of int): scaled bounding box
    """

    xmin,xmax,ymin,ymax,zmin,zmax = np.array(bbox_3d(vol))
    vol_in_bbox = vol[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1]

    new_shape = np.array(vol_in_bbox.shape) * rescale_factor
    new_vol_in_bbox = resize(vol_in_bbox, new_shape) > 0

    new_xmin, new_ymin, new_zmin = np.array((xmin,ymin,zmin)) * rescale_factor
    new_xmax = new_xmin + new_vol_in_bbox.shape[1] - 1
    new_ymax = new_ymin + new_vol_in_bbox.shape[0] - 1
    new_zmax = new_zmin + new_vol_in_bbox.shape[2] - 1
    new_bbox = new_xmin, new_xmax, new_ymin, new_ymax, new_zmin, new_zmax
    
    return new_vol_in_bbox, new_bbox

In [3]:
classifier_id = 37
warp_setting = 8

for stack in ['MD652', 'MD653', 'MD657', 'MD658']:

    timestamp = datetime.datetime.now().strftime("%m%d%Y%H%M%S")

    warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m='atlasV3', stack_f=stack,
                                    classifier_setting_m=classifier_id,
                                    classifier_setting_f=classifier_id,
                                    warp_setting=warp_setting, sided=True)
    
    volume_downsample = 32
    annotation_structure_volume_downsample = 8

    level = .5

    structure_entries = {}

    for structure, vol in warped_volumes.iteritems():
        print structure

        binary_vol = vol > level
        new_vol, new_bbox = rescale_binary_volume(binary_vol, volume_downsample / annotation_structure_volume_downsample)
    #     new_vol, new_bbox = rescale_binary_volume(binary_vol, 1)

        name_u, side, _, _ = parse_label(structure)
        if side is None:
            side = 'S'

        structure_id = str(uuid.uuid4().fields[-1])
        structure_entry = {
            'name': name_u,
            'side': side,
            'volume_in_bbox': new_vol,
            'bbox': new_bbox,
            'downsample': annotation_structure_volume_downsample,
            'creator': 'auto',
            'time_created': timestamp,
            'edits': []
        }
        structure_entries[structure_id] = structure_entry

    structures_df = DataFrame(structure_entries).T
    structures_df_fp = DataManager.get_annotation_filepath(stack=stack, by_human=False, stack_m='atlasV3', 
                                                           classifier_setting_m=classifier_id,
                                                          classifier_setting_f=classifier_id,
                                                          warp_setting=warp_setting, 
                                                           suffix='structures', timestamp=timestamp)
    save_hdf_v2(structure_entries, structures_df_fp)
    upload_to_s3(structures_df_fp)   

rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_10N_R.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes
aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_10N_R.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_10N_R.bp


Child returned 0
Child returned 1
0.79 seconds.
File does not exist: /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_10N_R.bp
[Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_10N_R.bp'
Score volume for 10N_R does not exist.


rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_sp5.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes


Child returned 0
Child returned 1
0.86 seconds.
File does not exist: /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_sp5.bp
[Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_sp5.bp'
Score volume for sp5 does not exist.


aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_sp5.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_sp5.bp
rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37_outerContour.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD652_down32_scoreVolume_clf_37/score_volumes


...

Child returned 0


aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37_outerContour.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37_outerContour.bp


Child returned 1
1.57 seconds.
File does not exist: /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37_outerContour.bp
[Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37_outerContour.bp'
Score volume for outerContour does not exist.


LRt_R
Sp5C_L
Pn_L
7N_R
PBG_L
VLL_L
4N_R
VLL_R
7N_L
PBG_R
RtTg
Sp5C_R
Pn_R
LRt_L
Sp5O_R
SNR_L
12N
SNR_R
Sp5O_L
DC_R
VCA_L
3N_L
6N_R
Sp5I_L
RMC_R
5N_L
SNC_L
SNC_R
AP
RMC_L
Sp5I_R
3N_R
6N_L
IC
VCA_R
DC_L
Amb_R
VCP_R
LC_R
10N_R
7n_R
5N_R
Tz_L
Tz_R
7n_L
4N_L
LC_L
10N_L
SC
VCP_L
Amb_L
aws s3 cp /shared/CSHL_labelings_v3/MD658/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37_structures_06012017100014.hdf s3://mousebrainatlas-data/CSHL_labelings_v3/MD658/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD658_down32_scoreVolume_clf_37_structures_06012017100014.hdf


Child returned 0
3.59 seconds.


In [48]:
# r = downsample_factor / structure_volume_downsample
# new_vol_in_bbox = rescale(vol_in_bbox, (r,r,r))

In [ ]:
# Load annotations created by our system.
structures_df = DataManager.load_annotation_v3(stack_m='atlasV3', by_human=False, 
                                                      stack=stack, 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                                           suffix='structures', timestamp=timestamp)

In [ ]:
# structure_entries = {}
# for s in all_known_structures_sided:
#     name_u, side, _, _ = parse_label(s)
#     if side is None:
#         side = 'S'

#     contour_points_grouped_by_pos = {np.mean(DataManager.convert_section_to_z(stack=stack, sec=p['section'], downsample=1)) / volume_downsample: 
#                                      p['vertices'] / volume_downsample
#                                      for _, p in contours_df[(contours_df['name'] == name_u) & (contours_df['side'] == side)].iterrows()}

#     if len(contour_points_grouped_by_pos) >= 2:
#         vol, bbox = interpolate_contours_to_volume(contour_points_grouped_by_pos, 'z')
#         id_ = str(uuid.uuid4().fields[-1])
#         structure_entry = {
#             'name': name_u,
#             'side': side,
#             'volume_in_bbox': vol,
#             'bbox': bbox,
#             'downsample': volume_downsample,
#             'creator': 'auto',
#             'time_created': datetime.datetime.now().strftime("%m%d%Y%H%M%S"),
#             'edits': []
#         }
#         structure_entries[id_] = structure_entry
#     else:
#         sys.stderr.write('Cannot interpolate because there are fewer than two contours for structure (%s, %s).\n' % (name_u, side))

# structures_df = DataFrame(structure_entries).T

# structures_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
#                                                        classifier_setting_m=classifier_setting_m,
#                                                       classifier_setting_f=classifier_setting_f,
#                                                       warp_setting=warp_setting, suffix='structures', timestamp=timestamp)
# save_hdf_v2(structure_entries, structures_df_fp)
# upload_to_s3(structures_df_fp)   

In [26]:
# Load annotations created by human
contours_df, _ = DataManager.load_annotation_v3(stack='MD594', by_human=True)

'No object named structures in the file'


Annotation has no structures.


In [5]:
# Load annotations created by our system.
contours_df, structures_df = DataManager.load_annotation_v3(stack_m='atlasV3', by_human=False, 
                                                      stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting)

In [6]:
print sorted(contours_df[contours_df['name'] == '4N']['section'])

[192, 193, 194, 195, 196, 197, 198, 214, 216, 217, 218, 219, 220, 221]


In [8]:
print metadata_cache['valid_sections']['MD635']

[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286

# Generate structure annotation to grid index lookup

In [15]:
from learning_utilities import generate_annotation_to_grid_indices_lookup
grid_index_lookup = generate_annotation_to_grid_indices_lookup(stack_m='atlasV3', by_human=False, stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                                               surround_margins=[500])

rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_masks && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_masks /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_masks


Child returned 0
Child returned 0
2.79 seconds.
Computing grid indices lookup for section 77...
Computing grid indices lookup for section 78...
Computing grid indices lookup for section 79...
Computing grid indices lookup for section 80...
Computing grid indices lookup for section 81...
Computing grid indices lookup for section 82...
Computing grid indices lookup for section 83...
Computing grid indices lookup for section 84...
Computing grid indices lookup for section 85...
Computing grid indices lookup for section 86...
Computing grid indices lookup for section 87...
Computing grid indices lookup for section 88...
Computing grid indices lookup for section 89...
Computing grid indices lookup for section 90...
Computing grid indices lookup for section 91...
Computing grid indices lookup for section 92...
Computing grid indices lookup for section 94...
Computing grid indices lookup for section 95...
Computing grid indices lookup for section 96...
Computing grid indices lookup for sectio

Computing grid indices lookup for section 251...
Computing grid indices lookup for section 252...
Computing grid indices lookup for section 253...
Computing grid indices lookup for section 254...
Computing grid indices lookup for section 255...
Computing grid indices lookup for section 256...
Computing grid indices lookup for section 257...
Computing grid indices lookup for section 258...
Computing grid indices lookup for section 259...
Computing grid indices lookup for section 260...
Computing grid indices lookup for section 261...
Computing grid indices lookup for section 262...
Computing grid indices lookup for section 263...
Computing grid indices lookup for section 264...
Computing grid indices lookup for section 265...
Computing grid indices lookup for section 266...
Computing grid indices lookup for section 267...
Computing grid indices lookup for section 268...
Computing grid indices lookup for section 269...
Computing grid indices lookup for section 270...
Computing grid indic

In [21]:
grid_indices_lookup_fp = DataManager.get_annotation_to_grid_indices_lookup_filepath(stack_m='atlasV3', 
                                                                                    by_human=False, stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting)
save_hdf_v2(grid_index_lookup, grid_indices_lookup_fp)
upload_to_s3(grid_indices_lookup_fp)

aws s3 cp /shared/CSHL_labelings_v3/MD635/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_grid_indices_lookup.hdf s3://mousebrainatlas-data/CSHL_labelings_v3/MD635/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_grid_indices_lookup.hdf


Child returned 0
3.84 seconds.


# Upsample structure volumes

In [13]:
# Load annotations created by our system.
contours_df, _ = DataManager.load_annotation_v3(stack_m='atlasV3', by_human=False, 
                                                      stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting)

In [15]:
volume_downsample = 8

In [ ]:
for s in all_known_structures_sided:
    name_u, side, _, _ = parse_label(s)
    contour_points_grouped_by_pos = {np.mean(DataManager.convert_section_to_z(stack='MD635', sec=p['section'], downsample=1)) / volume_downsample: 
                                     p['vertices'] / volume_downsample
                                     for _, p in contours_df[(contours_df['name'] == name_u) & (contours_df['side'] == side)].iterrows()}
    
    if len(contour_points_grouped_by_pos) >= 2:
        vol, bbox = interpolate_contours_to_volume(contour_points_grouped_by_pos, 'z')        
    else:
        sys.stderr.write('Cannot interpolate because there are fewer than two contours for structure %s.\n' % (name_u, side))